In [2]:
import model
import pandas as pd
import train
import numpy as np
import os

In [57]:
sub_df = pd.read_pickle("./sub_df_droped.pickle")
train_df = pd.read_pickle("./train_df_droped.pickle")


In [8]:
train_df.to_csv('./train_data/train.csv', index=False)
train_df.to_csv('./train_data/val.csv', index=False)
sub_df.to_csv('./train_data/predict.csv', index=False)


In [86]:
train_df.columns

Index(['weekday', 'holiday_flg', 'transaction', 'merchant_id', 'store_id',
       'store_merchant_id', 'active_user', 'GMV', 'category_name',
       'sub_category_name', 'prefecture', 'scan', 'weather', 'humidity',
       'rain', 'Month', 'Day', 'date', 'prefecture_area',
       'store_merchant_id_count', 'sub_category_name_count',
       'transaction_group_win_mean', 'GMV_group_win_mean',
       'active_user_group_win_mean', 'store_merchant_idtransactionmean',
       'store_merchant_idtransactionstd', 'store_merchant_idGMVmean',
       'store_merchant_idGMVstd', 'store_merchant_idactive_usermean',
       'store_merchant_idactive_userstd'],
      dtype='object')

In [39]:
zero_stores = train_df[(train_df.transaction == 0) & (train_df.weekday == 1)].store_merchant_id.unique()

In [67]:
sub_df.weekday = 1
sub_df.to_pickle('archive/sub_df')

In [68]:
sub_df = pd.read_pickle('archive/sub_df')
train_df = pd.read_pickle('archive/train_df')

import pickle
import datetime as dt
import numpy as np
with open('archive/columns', 'rb') as f:
    columns = pickle.load(f)
    
train_end = train_df.date.searchsorted(dt.datetime(2019, 9, 27))
train_df[columns].iloc[:train_end].to_csv('./train_data/train.csv', index=False)
train_df[columns].iloc[train_end:].to_csv('./train_data/val.csv', index=False)
sub_df[columns].to_csv('./train_data/predict.csv', index=False)
sub_df.transaction = np.nan

In [69]:
model_instance = model.lightGBM('./train_data')
model_instance.train(nround=700)
model_instance.get_importance()

[1]	valid_0's rmse: 23.1533
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 21.4197
[3]	valid_0's rmse: 19.8907
[4]	valid_0's rmse: 18.4931
[5]	valid_0's rmse: 17.2908
[6]	valid_0's rmse: 16.2289
[7]	valid_0's rmse: 15.2751
[8]	valid_0's rmse: 14.4618
[9]	valid_0's rmse: 13.726
[10]	valid_0's rmse: 13.0368
[11]	valid_0's rmse: 12.4233
[12]	valid_0's rmse: 11.8958
[13]	valid_0's rmse: 11.3731
[14]	valid_0's rmse: 10.9149
[15]	valid_0's rmse: 10.5143
[16]	valid_0's rmse: 10.1863
[17]	valid_0's rmse: 9.87217
[18]	valid_0's rmse: 9.56993
[19]	valid_0's rmse: 9.34216
[20]	valid_0's rmse: 9.1309
[21]	valid_0's rmse: 8.91683
[22]	valid_0's rmse: 8.75356
[23]	valid_0's rmse: 8.57286
[24]	valid_0's rmse: 8.42672
[25]	valid_0's rmse: 8.28219
[26]	valid_0's rmse: 8.14083
[27]	valid_0's rmse: 8.02456
[28]	valid_0's rmse: 7.94
[29]	valid_0's rmse: 7.83759
[30]	valid_0's rmse: 7.73684
[31]	valid_0's rmse: 7.65398
[32]	valid_0's rmse: 7.56713
[33]	valid_0's rmse: 7.511

,importance
weekday,416
category_name,68
sub_category_name,281
prefecture,181
scan,14
humidity,450
rain,130
weather,84
prefecture_area,118
sub_category_name_count,85


In [114]:
sub_df['transaction'] = model_instance.predict()
sub_df.loc[sub_df.store_merchant_id.isin(zero_stores3), 'transaction'] = 0
sub_df.head()

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,...,Year,Month,Day,transaction_group_win_mean,store_merchant_idtransactionmean,store_merchant_idtransactionstd,store_merchant_idGMVmean,store_merchant_idGMVstd,store_merchant_idactive_usermean,store_merchant_idactive_userstd
0,2019-09-30,0,14.680148,1,350,0,2,11,1,566,...,2019,9,30,11.25,12.620690,5.038522,35501.758621,33669.271302,11.655172,4.669627
1,2019-09-30,1,0.187563,1,3,1,2,17,10,50,...,2019,9,30,0.75,0.344828,0.669534,5272.379310,19187.950930,0.310345,0.541390
2,2019-09-30,2,8.881459,1,65,2,2,4,9,167,...,2019,9,30,9.25,7.931034,2.737713,5278.655172,2271.290276,7.517241,2.707406
3,2019-09-30,3,0.890492,1,564,3,2,26,5,237,...,2019,9,30,0.25,0.482759,0.687682,67.586207,94.515878,0.413793,0.568032
4,2019-09-30,4,44.534239,1,179,4,2,24,9,683,...,2019,9,30,64.75,53.103448,17.879247,120139.482759,56617.591665,49.482759,15.527903


In [101]:
sub_df[sub_df.store_merchant_id == 33359]

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,...,Year,Month,Day,transaction_group_win_mean,store_merchant_idtransactionmean,store_merchant_idtransactionstd,store_merchant_idGMVmean,store_merchant_idGMVstd,store_merchant_idactive_usermean,store_merchant_idactive_userstd
33359,2019-09-30,33359,1.388704,1,489,31922,2,17,5,237,...,2019,9,30,1.75,1.482759,1.703372,12750.758621,24669.630342,1.37931,1.449478


In [115]:
prediction_result = sub_df.transaction
get_submission(prediction_result, output='submission02_nonzero2.csv')

In [84]:

def get_submission(predict_series,csv_path = "./data/sample_submit.csv", output="./submit_team9.csv"):

    sub = pd.read_csv(csv_path)

    sub["transaction"]= predict_series

    sub.to_csv(output,index =False)

In [90]:
from sklearn.metrics import mean_squared_error


In [87]:
!cp train_data/predict.csv train_data/predict.csv.backup

In [92]:
!cp train_data/predict.csv.backup train_data/predict.csv

In [93]:
!cp -r train_data train_data2

In [94]:
!cp train_data2/val.csv train_data2/predict.csv

In [95]:
model_instance2  = model.lightGBM('train_data2')

In [96]:
model_instance2.train(nround=600)

[1]	valid_0's rmse: 23.1533
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 21.4197
[3]	valid_0's rmse: 19.8907
[4]	valid_0's rmse: 18.4931
[5]	valid_0's rmse: 17.2908
[6]	valid_0's rmse: 16.2289
[7]	valid_0's rmse: 15.2751
[8]	valid_0's rmse: 14.4618
[9]	valid_0's rmse: 13.726
[10]	valid_0's rmse: 13.0368
[11]	valid_0's rmse: 12.4233
[12]	valid_0's rmse: 11.8958
[13]	valid_0's rmse: 11.3731
[14]	valid_0's rmse: 10.9149
[15]	valid_0's rmse: 10.5143
[16]	valid_0's rmse: 10.1863
[17]	valid_0's rmse: 9.87217
[18]	valid_0's rmse: 9.56993
[19]	valid_0's rmse: 9.34216
[20]	valid_0's rmse: 9.1309
[21]	valid_0's rmse: 8.91683
[22]	valid_0's rmse: 8.75356
[23]	valid_0's rmse: 8.57286
[24]	valid_0's rmse: 8.42672
[25]	valid_0's rmse: 8.28219
[26]	valid_0's rmse: 8.14083
[27]	valid_0's rmse: 8.02456
[28]	valid_0's rmse: 7.94
[29]	valid_0's rmse: 7.83759
[30]	valid_0's rmse: 7.73684
[31]	valid_0's rmse: 7.65398
[32]	valid_0's rmse: 7.56713
[33]	valid_0's rmse: 7.511

In [104]:
np.sqrt(mean_squared_error(model_instance2.predict(), train_df[columns].iloc[train_end:].transaction))

6.210055802870894

In [118]:
temp = train_df.iloc[train_end:]
temp.transaction = model_instance2.predict()
temp.loc[temp.store_merchant_id.isin(zero_stores3), 'transaction'] = 0
np.sqrt(mean_squared_error(temp.transaction, train_df[columns].iloc[train_end:].transaction))

/home/ec2-user/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/ec2-user/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


6.211532433539811

In [113]:
meanlist = train_df[train_df.weekday == 1].groupby('store_merchant_id').transaction.mean()
zero_stores3 = meanlist[meanlist == 0].index

In [116]:
len(zero_stores3)

2264